In [2]:
import datashader as ds
import datashader.transfer_functions as tf
import numpy as np
import pandas as pd
from IPython.core.display import HTML, display

In [3]:
data = pd.read_csv('../../data/external/2006 - RENACOM.csv' , encoding = "ISO-8859-1")

In [4]:
data['LONGITUDE']  = pd.to_numeric(data['LONGITUDE'] , errors='coerce')
data['LATITUDE'] = pd.to_numeric(data['LATITUDE'] , errors='coerce')

In [5]:
import pyproj

wgs84=pyproj.Proj("+init=EPSG:4326")
webmercator = pyproj.Proj("+init=EPSG:3857")

data['meterswest'], data['metersnorth'] = pyproj.transform(wgs84, webmercator, list(data['LONGITUDE']) , list(data['LATITUDE']))

In [6]:
display(HTML("<style>.container { width:100% !important; }</style>"))

In [7]:
background = "black"
plot_width  = int(1000)
plot_height = int(plot_width*7.0/12)

Niger = ((20400,  1603000), (1309690, 2403810))
x_range,y_range = Niger

In [8]:
from functools import partial
from datashader.utils import export_image
from datashader.colors import colormap_select, Greys9, Hot, viridis, inferno
from IPython.core.display import HTML, display

export = partial(export_image, background = background)
cm = partial(colormap_select, reverse=(background!="black"))

In [9]:
cvs = ds.Canvas(plot_width, plot_height, *Niger)
agg = cvs.points(data, 'meterswest', 'metersnorth')

In [11]:
#if background == "black":
#      color_key = {'CR':'aqua', 'VA':'lime',  'CU':'red', 'PE':'fuchsia', 'QT':'yellow' }
#else: color_key = {'w':'blue', 'b':'green', 'a':'red', 'h':'orange',  'o':'saddlebrown'}

if background == "black":
      color_key = {'CR':'aqua',  'CU':'red' }
else: color_key = {'w':'blue', 'b':'green', 'a':'red', 'h':'orange',  'o':'saddlebrown'}

In [14]:
data.TYPECOM = data.TYPECOM.astype('category')

In [17]:
import bokeh.plotting as bp
from bokeh.models.tiles import WMTSTileSource

bp.output_file('test.html')

def base_plot(tools='pan,wheel_zoom,reset',webgl=False):
    p = bp.figure(tools=tools,
         plot_width=int(900), plot_height=int(500),
         x_range=x_range, y_range=y_range, outline_line_color=None,
         min_border=0, min_border_left=0, min_border_right=0,
         min_border_top=0, min_border_bottom=0, webgl=webgl)
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    return p

p = base_plot()

url="http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.png"
#url="http://tile.stamen.com/toner-background/{Z}/{X}/{Y}.png"
#p.yaxis.major_label_orientation = 'vertical'
tile_renderer = p.add_tile(WMTSTileSource(url=url))
tile_renderer.alpha=1.0 #if background == "black" else 0.15


In [19]:
from datashader.bokeh_ext import InteractiveImage

def image_callback(x_range, y_range, w, h):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(data, 'meterswest', 'metersnorth', ds.count_cat('TYPECOM'))
    img = tf.colorize(agg, color_key, 'log')
    return tf.dynspread(img,threshold=0.75, max_px=8)

InteractiveImage(p, image_callback)